In [1]:
import os
import json
import torch
import pandas as pd
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
from timm.models.vision_transformer import VisionTransformer
from torchvision.models.resnet import Bottleneck, ResNet
import numpy as np

class MultiModalSurvivalDataset(Dataset):
    def __init__(self, base_dir="cases", embedding_modalities=None, methylation_modalities=None, other_data_modalities=None, one_to_one=True, include_clinical_staging=True):
        self.metadata = []
        self.missing_counts = {modality: 0 for modality in embedding_modalities} if embedding_modalities else {}
        self.missing_methylation_counts = {modality: 0 for modality in methylation_modalities} if methylation_modalities else {}
        self.missing_other_data_counts = {modality: 0 for modality in other_data_modalities} if other_data_modalities else {}
        self.missing_survival_data_count = 0
        self.failed_cases = {}

        self.slide_stats_df = pd.read_csv(f"{base_dir}/slide_statistics.csv")
        case_dirs = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and d != "GENERAL_METADATA"]

        for case_id in case_dirs:
            case_dir = os.path.join(base_dir, case_id)
            metadata_path = os.path.join(case_dir, "aggregated_data", f'{case_id}_data.json')

            if not os.path.exists(metadata_path):
                print(f"No metadata found for case {case_id}, skipping...")
                continue

            with open(metadata_path, 'r') as f:
                case_metadata = json.load(f)

            if not all(case_metadata.get(key, {}).get("has_data", False) for key in ["biospecimen", "clinical", "methylation"]):
                self.failed_cases[case_id] = self.failed_cases.get(case_id, []) + ["Missing required data"]
                continue

            for sample in case_metadata["biospecimen"]["biospecimen_data"]:
                if sample["sample_type"] == "Primary Tumor":
                    for slide in sample["slides"]:
                        case_sample_added = False
                        slide_barcode = slide["slide_barcode"]
                        
                        if slide["resolution_level"] <= 2:
                            continue
                        
                        survival_time = case_metadata["clinical"]["clinical_patient_data"].get("days_to_death")
                        event = 1
                        if survival_time is None:
                            survival_time = case_metadata["clinical"]["clinical_patient_data"].get("days_to_last_followup")
                            event = 0

                        if survival_time is None or (event == 0 and float(survival_time) == 0):
                            self.missing_survival_data_count += 1
                            continue
                        
                        sample_data = {"survival_time": float(survival_time), "event": event}

                        # WSI Slide Embedding Data
                        for modality in embedding_modalities:
                            embedding_key = f"embedding_{modality}"
                            if embedding_key in slide and slide[embedding_key] is not None:
                                sample_data[modality] = slide[embedding_key]
                            else:
                                self.missing_counts[modality] += 1
                                
                        # Other Data Types (e.g., Methylation, Clinical)
                        for model_name in methylation_modalities:
                            methylation_data = self.extract_methylation_embedding(case_metadata, model_name)
                            if methylation_data is not None:
                                sample_data[model_name] = methylation_data
                            else:
                                self.missing_methylation_counts[model_name] += 1

                        if "clinical" in other_data_modalities:
                            clinical_data = self.extract_clinical_features(case_metadata, include_clinical_staging)
                            if clinical_data is not None:
                                sample_data["clinical"] = clinical_data
                            else:
                                self.missing_other_data_counts["clinical"] += 1

                        if len(sample_data) == len(embedding_modalities) + len(methylation_modalities) + len(other_data_modalities) + 2:
                            self.metadata.append(sample_data)
                            case_sample_added = True
                        else:
                            self.failed_cases[case_id] = self.failed_cases.get(case_id, []) + ["Incomplete data"]
                        
                        if one_to_one and case_sample_added:
                            break
                        
        # if self.failed_cases:
        #     print("Failed cases and reasons:")
        #     for case_id, errors in self.failed_cases.items():
        #         print(f"Case {case_id} failed due to: {', '.join(errors)}")
        #     else:
        #         print("All cases processed successfully.")

    def extract_methylation_embedding(self, case_metadata, model_name):
        """Extract methylation embedding if available."""
        methylation_data = case_metadata.get("methylation", {}).get("embeddings", [])
        
        for entry in methylation_data:
                for embedding_entry in entry.get("embeddings", []):
                    if embedding_entry.get("model_name") == model_name and embedding_entry.get("embedding") is not None:
                        return embedding_entry["embedding"]
            
        return None

    def extract_clinical_features(self, case_metadata, include_clinical_staging):
        """Extract and encode clinical features."""
        clinical_data = case_metadata.get("clinical", {}).get("clinical_patient_data", {})

        features = {
            "age": self.transform_age(clinical_data.get("days_to_birth", 0)),
            "gender": self.encode_gender(clinical_data.get("gender", "unknown"))
        }

        if include_clinical_staging:
            features.update({
                "tumor_stage": self.encode_tumor_stage(clinical_data.get("pathologic_stage", "unknown")),
                "tumor_site": self.encode_tumor_site(clinical_data.get("tumor_tissue_site", "unknown")),
                "ajcc_stage": self.encode_ajcc_stage(clinical_data.get("ajcc_stage_version", "unknown")),
                "pathologic_T": self.encode_pathologic_stage(clinical_data.get("pathologic_T", "unknown")),
                "pathologic_N": self.encode_pathologic_stage(clinical_data.get("pathologic_N", "unknown")),
                "pathologic_M": self.encode_pathologic_stage(clinical_data.get("pathologic_M", "unknown"))
            })

        return torch.tensor(list(features.values()), dtype=torch.float32)

    def transform_age(self, days_to_birth):
        """Convert days_to_birth to age. Return 0 if invalid."""
        try:
            age = abs(int(days_to_birth) / 365)
            return age
        except:
            return 0

    def encode_gender(self, gender):
        """Encode gender as 0 for Male, 1 for Female, or -1 for unknown."""
        gender_map = {"MALE": 0, "FEMALE": 1}
        return gender_map.get(gender.upper(), -1)

    def encode_tumor_stage(self, tumor_stage):
        """Encode tumor stage as categorical values."""
        tumor_stage_map = {
            "Stage I": 0, "Stage IIA": 1, "Stage IIB": 2, "Stage IIIA": 3,
            "Stage IIIB": 4, "Stage IIIC": 5, "Stage IV": 6, "unknown": -1
        }
        return tumor_stage_map.get(tumor_stage, -1)

    def encode_tumor_site(self, tumor_site):
        """Encode tumor tissue site as categorical values."""
        tumor_site_map = {
            "Colon": 0, "Lung": 1, "Breast": 2, "Prostate": 3, "unknown": -1
        }
        return tumor_site_map.get(tumor_site, -1)

    def encode_ajcc_stage(self, ajcc_stage):
        """Encode AJCC stage as categorical values."""
        ajcc_stage_map = {
            "7th": 0, "8th": 1, "unknown": -1
        }
        return ajcc_stage_map.get(ajcc_stage, -1)

    def encode_pathologic_stage(self, pathologic_stage):
        """Encode pathologic stage (T, N, M) as categorical values."""
        pathologic_map = {
            "T1": 0, "T2": 1, "T3": 2, "T4": 3,
            "N1": 0, "N2": 1, "N3": 2,
            "M0": 0, "M1": 1, "unknown": -1
        }
        return pathologic_map.get(pathologic_stage, -1)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, index):
        item = self.metadata[index]
        embeddings = {
            modality: item[modality].clone().detach().to(torch.float32) 
            if isinstance(item[modality], torch.Tensor) 
            else torch.tensor(item[modality], dtype=torch.float32)
            for modality in item if modality not in ["survival_time", "event"]
        }
        return (
            embeddings, 
            torch.tensor(item["survival_time"], dtype=torch.float32), 
            torch.tensor(item["event"], dtype=torch.float32)
        )


In [2]:
embedding_modalities = ["vit_DINO", "resnet50", "UNI_v1"]
methylation_modalities = ["ae_normalAE", "VAE_100k"]
other_data_modalities = ["clinical"]

dataset = MultiModalSurvivalDataset(base_dir="cases", 
                                   embedding_modalities=embedding_modalities,
                                   methylation_modalities=methylation_modalities,
                                   other_data_modalities=other_data_modalities,
                                   one_to_one=True,
                                   include_clinical_staging=False)

print(f"Total samples in dataset: {len(dataset)}\n")

count = 0
average_survival = 0
survival_times = []

for i in range(len(dataset)):
    embeddings, survival_time, event = dataset[i]
    #print(f"Sample {i+1}:")
    #for modality, tensor in embeddings.items():
        #print(f"  {modality} embedding shape: {tensor.shape}")
    #print(f"  Survival Time: {survival_time.item()}, Event: {event.item()}\n")
    if(survival_time.item() == 0):
        print(f"Sample {i+1}:")
        print(f"  Survival Time: {survival_time.item()}, Event: {event.item()}\n")
    survival_times.append(survival_time.item())
    average_survival += survival_time.item()
    count += event.item()

print("Total event:", count)
print("Average Survival:", average_survival/len(dataset))
print("Median Survival:", np.median(survival_times))

Total samples in dataset: 274

Sample 272:
  Survival Time: 0.0, Event: 1.0

Total event: 41.0
Average Survival: 552.4087591240876
Median Survival: 221.0


In [3]:
embedding_modalities = ["vit_DINO", "resnet50", "UNI_v1"]
methylation_modalities = ["ae_normalAE", "VAE_100k"]
other_data_modalities = ["clinical"]

dataset = MultiModalSurvivalDataset(base_dir="cases", 
                                   embedding_modalities=embedding_modalities,
                                   methylation_modalities=methylation_modalities,
                                   other_data_modalities=other_data_modalities,
                                   one_to_one=False,
                                   include_clinical_staging=False)

print(f"Total samples in dataset: {len(dataset)}\n")

count = 0
average_survival = 0
survival_times = []

for i in range(len(dataset)):
    embeddings, survival_time, event = dataset[i]
    #print(f"Sample {i+1}:")
    #for modality, tensor in embeddings.items():
        #print(f"  {modality} embedding shape: {tensor.shape}")
    #print(f"  Survival Time: {survival_time.item()}, Event: {event.item()}\n")
    if(survival_time.item() == 0):
        print(f"Sample {i+1}:")
        print(f"  Survival Time: {survival_time.item()}, Event: {event.item()}\n")
    survival_times.append(survival_time.item())
    average_survival += survival_time.item()
    count += event.item()

print("Total event:", count)
print("Average Survival:", average_survival/len(dataset))
print("Median Survival:", np.median(survival_times))

Total samples in dataset: 459

Sample 454:
  Survival Time: 0.0, Event: 1.0

Sample 455:
  Survival Time: 0.0, Event: 1.0

Total event: 63.0
Average Survival: 511.42483660130716
Median Survival: 212.0


In [ ]:
import numpy as np
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv
from sklearn.model_selection import train_test_split
from sksurv.metrics import concordance_index_censored

# embedding_modalities = ["vit_DINO", "resnet50", "UNI_v2"]
# methylation_modalities = ["ae_normalAE", "VAE_100k"]
# other_data_modalities = ["clinical"]

embedding_modalities = ["vit_DINO"]
methylation_modalities = ["VAE_100k"]
other_data_modalities = []

dataset = MultiModalSurvivalDataset(base_dir="cases", 
                                   embedding_modalities=embedding_modalities,
                                   methylation_modalities=methylation_modalities,
                                   other_data_modalities=other_data_modalities,
                                   one_to_one=True,
                                   include_clinical_staging=False)

features = []
survival_times = []
events = []

for i in range(len(dataset)):
    embeddings, survival_time, event = dataset[i]

    flattened_embeddings = np.concatenate([embeddings[modality].numpy().flatten() for modality in embeddings])
    
    features.append(flattened_embeddings)
    survival_times.append(survival_time.item())
    events.append(event.item())

features = np.array(features)
survival_times = np.array(survival_times)
events = np.array(events)

survival_data = Surv.from_arrays(event=events, time=survival_times)
X_train, X_test, y_train, y_test = train_test_split(features, survival_data, test_size=0.2, random_state=42)

model = GradientBoostingSurvivalAnalysis(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

risk_scores_train = model.predict(X_train)
risk_scores_test = model.predict(X_test)

c_index_train = concordance_index_censored(y_train['event'], y_train['time'], risk_scores_train)
c_index_test = concordance_index_censored(y_test['event'], y_test['time'], risk_scores_test)

print(f"Train Concordance Index: {c_index_train[0]}")
print(f"Test Concordance Index: {c_index_test[0]}")

# Run Experiments (no HPO)

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import shutil

def load_dataset(embedding_modalities, methylation_modalities, other_data_modalities, one_to_one, include_clinical_staging, base_dir="cases"):
    """
    Load the dataset based on the given modalities.
    """
    return dataset = MultiModalSurvivalDataset(base_dir=base_dir, 
                                    embedding_modalities=embedding_modalities,
                                    methylation_modalities=methylation_modalities,
                                    other_data_modalities=other_data_modalities,
                                    one_to_one=one_to_one,
                                    include_clinical_staging=include_clinical_staging)

def clean_csv(folder_name, csv_path="results/results.csv"):
    """
    Remove any rows from the CSV that correspond to the same folder_name.
    """
    if os.path.exists(csv_path):
        results_df = pd.read_csv(csv_path)
        results_df = results_df[results_df['experiment_name'] != folder_name]
        results_df.to_csv(csv_path, index=False)

def bootstrap_c_index(model, X, y, n_bootstraps=25):
    """
    Compute bootstrapped concordance index.
    """
    c_indices = []
    n_samples = len(y)
    for _ in range(n_bootstraps):
        indices = np.random.choice(n_samples, n_samples, replace=True)
        c_index, *_ = concordance_index_censored(y['event'][indices], y['time'][indices], model.predict(X[indices]))
        c_indices.append(c_index)
    return np.mean(c_indices), np.percentile(c_indices, 2.5), np.percentile(c_indices, 97.5)

def plot_km_curve(y, risk_scores, output_dir):
    """
    Plot Kaplan-Meier curve for high-risk and low-risk groups.
    """
    median_risk = np.median(risk_scores)
    high_risk_idx = risk_scores >= median_risk
    low_risk_idx = risk_scores < median_risk
    
    kmf_high = KaplanMeierFitter()
    kmf_low = KaplanMeierFitter()
    
    kmf_high.fit(y['time'][high_risk_idx], event_observed=y['event'][high_risk_idx], label="High Risk")
    kmf_low.fit(y['time'][low_risk_idx], event_observed=y['event'][low_risk_idx], label="Low Risk")
    
    results = logrank_test(y['time'][high_risk_idx], y['time'][low_risk_idx],
                           event_observed_A=y['event'][high_risk_idx], event_observed_B=y['event'][low_risk_idx])
    
    plt.figure(figsize=(8, 6))
    kmf_high.plot(ci_show=False, color='blue')
    kmf_low.plot(ci_show=False, color='red')
    
    plt.title(f"Kaplan-Meier Curves\nLog-rank p-value: {results.p_value:.5f}")
    plt.xlabel("Time (days)")
    plt.ylabel("Survival Probability")
    plt.legend()
    plt.grid(True)
    
    km_plot_path = os.path.join(output_dir, "km_curve.png")
    plt.savefig(km_plot_path)
    plt.close()
    
    return km_plot_path, results.p_value

def run_experiment(embedding_modalities, methylation_modalities, other_data_modalities, base_output_dir="results", results_csv="results.csv"):
    """
    Run the survival analysis experiment with the given modalities, save results, and update the CSV file.
    """
    dataset = load_dataset(embedding_modalities, methylation_modalities, other_data_modalities)

    folder_name = f"embeddings_{'_'.join(embedding_modalities)}_" \
                  f"methylation_{'_'.join(methylation_modalities)}_" \
                  f"clinical_{'_'.join(other_data_modalities)}"
    output_dir = os.path.join(base_output_dir, folder_name)

    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    features = []
    survival_times = []
    events = []

    # Prepare data
    for i in range(len(dataset)):
        embeddings, survival_time, event = dataset[i]
        flattened_embeddings = np.concatenate([embeddings[modality].numpy().flatten() for modality in embeddings])
        
        features.append(flattened_embeddings)
        survival_times.append(survival_time.item())
        events.append(event.item())

    features = np.array(features)
    survival_times = np.array(survival_times)
    events = np.array(events)

    # Survival data
    survival_data = Surv.from_arrays(event=events, time=survival_times)
    X_train, X_test, y_train, y_test = train_test_split(features, survival_data, test_size=0.2, random_state=42)

    # Train the model
    model = GradientBoostingSurvivalAnalysis(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
    model.fit(X_train, y_train)

    # Predict risk scores
    risk_scores_train = model.predict(X_train)
    risk_scores_test = model.predict(X_test)

    c_index_train, ci_lower_train, ci_upper_train = bootstrap_c_index(model, X_train, y_train)
    c_index_test, ci_lower_test, ci_upper_test = bootstrap_c_index(model, X_test, y_test)
    
    with open(os.path.join(output_dir, "concordance_index.txt"), "w") as f:
        f.write(f"Train Concordance Index: {c_index_train:.3f} ({ci_lower_train:.3f}-{ci_upper_train:.3f})\n")
        f.write(f"Test Concordance Index: {c_index_test:.3f} ({ci_lower_test:.3f}-{ci_upper_test:.3f})\n")
    
    km_plot_path, logrank_p = plot_km_curve(y_test, risk_scores_test, output_dir)
    
    return {
        'experiment_name': folder_name,
        'train_c_index': c_index_train,
        'test_c_index': c_index_test,
        'train_c_index_ci': (ci_lower_train, ci_upper_train),
        'test_c_index_ci': (ci_lower_test, ci_upper_test),
        'km_plot': km_plot_path,
        'logrank_p_value': logrank_p,
        'biospecimen_embeddings_used': '_'.join(embedding_modalities),
        'methylation_embeddings_used': '_'.join(methylation_modalities),
        'clinical_used': '_'.join(other_data_modalities)
    }

    clean_csv(folder_name)

    return experiment_result

def generate_experiments():
    results_dir = "results"
    os.makedirs(results_dir, exist_ok=True)
    
    # embedding_modalities = ["vit_DINO", "resnet50", "UNI_v2"]
    # methylation_modalities = ["ae_normalAE", "VAE_100k"]
    # other_data_modalities = ["clinical"]

    embedding_modalities = [[], ["vit_DINO"], ["resnet50"], ["UNI_v1"], ["vit_DINO", "resnet50"], ["vit_DINO", "UNI_v1"], ["resnet50", "UNI_v1"], ["vit_DINO", "resnet50", "UNI_v1"]]
    methylation_modalities = [[], ["ae_normalAE"], ["VAE_100k"], ["ae_normalAE", "VAE_100k"]]
    other_data_modalities = [[], ["clinical"]]

    all_results = []

    # Loop over combinations and run experiments
    for embedding in embedding_modalities:
        for methylation in methylation_modalities:
            for other_data in other_data_modalities:
                if(embedding == [] and methylation == [] and other_data == []):
                    continue
                experiment_result = run_experiment(embedding, methylation, other_data)
                all_results.append(experiment_result)

    results_df = pd.DataFrame(all_results)
    results_df = results_df.sort_values(by="test_c_index", ascending=False)
    results_df.to_csv(os.path.join(results_dir, "results.csv"), index=False)

    print(f"All experiments completed. Results saved to {os.path.join(results_dir, 'results.csv')}")

generate_experiments()

# Run Experiments (with HPO)

In [ ]:
import os
import re
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sksurv.metrics import concordance_index_censored
from sklearn.model_selection import train_test_split
from sksurv.ensemble import GradientBoostingSurvivalAnalysis
from sksurv.util import Surv
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
import shutil
import optuna
from tqdm.auto import tqdm

def objective(trial, X_train, X_test, y_train, y_test):
    """
    Objective function for hyperparameter optimization using Optuna.
    """
    # Define hyperparameter search space
    n_estimators = trial.suggest_int("n_estimators", 10, 100)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.15, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 5)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)

    # Train the survival model
    model = GradientBoostingSurvivalAnalysis(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        random_state=42
    )
    
    model.fit(X_train, y_train)
    c_index_test, *_ = bootstrap_c_index(model, X_test, y_test)
    
    return c_index_test

def load_dataset(embedding_modalities, methylation_modalities, other_data_modalities, one_to_one, include_clinical_staging, base_dir="cases"):
    """
    Load the dataset based on the given modalities.
    """
    return MultiModalSurvivalDataset(base_dir=base_dir, 
                                    embedding_modalities=embedding_modalities,
                                    methylation_modalities=methylation_modalities,
                                    other_data_modalities=other_data_modalities,
                                    one_to_one=one_to_one,
                                    include_clinical_staging=include_clinical_staging)

def extract_best_hyperparameters(concordance_file_path):
    """
    Extract the best hyperparameters from the existing concordance_index.txt file.
    """
    if not os.path.exists(concordance_file_path):
        return None

    with open(concordance_file_path, "r") as f:
        content = f.read()

    match = re.search(r"Best Hyperparameters: (.+)", content)
    if match:
        try:
            best_params = json.loads(match.group(1).replace("'", '"'))
            return best_params
        except json.JSONDecodeError:
            return None
    return None

def clean_csv(folder_name, csv_path="results/results.csv"):
    """
    Remove any rows from the CSV that correspond to the same folder_name.
    """
    if os.path.exists(csv_path):
        results_df = pd.read_csv(csv_path)
        results_df = results_df[results_df['experiment_name'] != folder_name]
        results_df.to_csv(csv_path, index=False)

def bootstrap_c_index(model, X, y, n_bootstraps=100, sample_fraction=0.5):
    """
    Compute bootstrapped concordance index.
    """
    c_indices = []
    
    uncensored_idx = y['event'] == 1
    censored_idx = ~uncensored_idx

    X_uncensored, y_uncensored = X[uncensored_idx], y[uncensored_idx]
    X_censored, y_censored = X[censored_idx], y[censored_idx]

    n_uncensored = len(X_uncensored)
    n_censored = len(X_censored)
    total_samples = n_uncensored + n_censored
    n_uncensored_sample = int(n_uncensored * sample_fraction)
    n_censored_sample = total_samples - n_uncensored_sample

    for _ in range(n_bootstraps):
        sampled_uncensored_idx = np.random.choice(n_uncensored, n_uncensored_sample, replace=True)
        sampled_censored_idx = np.random.choice(n_censored, n_censored_sample, replace=True)
        
        X_boot_uncensored, y_boot_uncensored = X_uncensored[sampled_uncensored_idx], y_uncensored[sampled_uncensored_idx]
        X_boot_censored, y_boot_censored = X_censored[sampled_censored_idx], y_censored[sampled_censored_idx]

        X_boot = np.concatenate([X_boot_uncensored, X_boot_censored])
        y_boot = np.concatenate([y_boot_uncensored, y_boot_censored])

        predictions = model.predict(X_boot)
        c_index, *_ = concordance_index_censored(y_boot['event'], y_boot['time'], model.predict(X_boot))
        c_indices.append(c_index)

    mean_c_index = np.mean(c_indices)
    ci_lower = np.percentile(c_indices, 2.5)
    ci_upper = np.percentile(c_indices, 97.5)

    return mean_c_index, ci_lower, ci_upper

def plot_km_curve(y, risk_scores, output_dir):
    """
    Plot Kaplan-Meier curve for high-risk and low-risk groups.
    """
    median_risk = np.median(risk_scores)
    high_risk_idx = risk_scores >= median_risk
    low_risk_idx = risk_scores < median_risk

    # Check if either group is empty and print additional details if so
    if np.sum(high_risk_idx) <= int(0.4*len(risk_scores)) or np.sum(low_risk_idx) <= int(0.4*len(risk_scores)):
        print("Error: One of the risk groups is outputting strange results.")
        print("Possible reasons include identical or nearly identical risk scores.")
        raise ValueError("One of the risk groups is messed up.")

    kmf_high = KaplanMeierFitter()
    kmf_low = KaplanMeierFitter()
    
    kmf_high.fit(y['time'][high_risk_idx], event_observed=y['event'][high_risk_idx], label="High Risk")
    kmf_low.fit(y['time'][low_risk_idx], event_observed=y['event'][low_risk_idx], label="Low Risk")
    
    results = logrank_test(y['time'][high_risk_idx], y['time'][low_risk_idx],
                           event_observed_A=y['event'][high_risk_idx], event_observed_B=y['event'][low_risk_idx])
    
    plt.figure(figsize=(8, 6))
    kmf_high.plot(ci_show=False, color='blue')
    kmf_low.plot(ci_show=False, color='red')
    
    plt.title(f"Kaplan-Meier Curves\nLog-rank p-value: {results.p_value:.5f}")
    plt.xlabel("Time (days)")
    plt.ylabel("Survival Probability")
    plt.legend()
    plt.grid(True)
    
    km_plot_path = os.path.join(output_dir, "km_curve.png")
    plt.savefig(km_plot_path)
    plt.close()
    
    return km_plot_path, results.p_value

def run_experiment(embedding_modalities, methylation_modalities, other_data_modalities, base_output_dir, test_size, one_to_one, include_clinical_staging, results_csv="results.csv"):
    """
    Run the survival analysis experiment with the given modalities, save results, and update the CSV file.
    """
    # Create output folder based on modalities
    folder_name = f"embeddings_{'_'.join(embedding_modalities)}_" \
                  f"methylation_{'_'.join(methylation_modalities)}_" \
                  f"clinical_{'_'.join(other_data_modalities)}"
    output_dir = os.path.join(base_output_dir, folder_name)
    concordance_file_path = os.path.join(output_dir, "concordance_index.txt")

    os.makedirs(output_dir, exist_ok=True)

    # Create dataset
    dataset = load_dataset(embedding_modalities, methylation_modalities, other_data_modalities, one_to_one=one_to_one, include_clinical_staging=include_clinical_staging)

    features = []
    survival_times = []
    events = []

    for i in range(len(dataset)):
        embeddings, survival_time, event = dataset[i]
        flattened_embeddings = np.concatenate([embeddings[modality].numpy().flatten() for modality in embeddings])
        
        features.append(flattened_embeddings)
        survival_times.append(survival_time.item())
        events.append(event.item())

    features = np.array(features)
    survival_times = np.array(survival_times)
    events = np.array(events)

    survival_data = Surv.from_arrays(event=events, time=survival_times)
    X_train, X_test, y_train, y_test = train_test_split(features, survival_data, test_size=test_size, random_state=42, stratify=events)

    best_params = extract_best_hyperparameters(concordance_file_path)

    if best_params is None:
        # **Hyperparameter Optimization**
        n_trials = 30
        study = optuna.create_study(direction="maximize")
        study.optimize(lambda trial: objective(trial, X_train, X_test, y_train, y_test), n_trials=n_trials, n_jobs=-1)
        best_params = study.best_params
    else:
        print("Using previous best hyperparams...")

    max_retries = 10
    retry_count = 0
    km_plot_path = None
    logrank_p = None

    while retry_count < max_retries:
        model = GradientBoostingSurvivalAnalysis(**best_params, random_state=42)
        model.fit(X_train, y_train)
        try:
            km_plot_path, logrank_p = plot_km_curve(y_test, model.predict(X_test), output_dir)
            break
        except ValueError:
            retry_count += 1
            print(f"Attempt {retry_count}: Plot failed due to constant risk scores. Re-running hyperparameter optimization...")
            study = optuna.create_study(direction="maximize")
            study.optimize(lambda trial: objective(trial, X_train, X_test, y_train, y_test), n_trials=30, n_jobs=-1)
            best_params = study.best_params
            if retry_count == max_retries:
                print("Maximum retries reached. Experiment failed.")
                return None

    # Compute final performance
    c_index_train, ci_lower_train, ci_upper_train = bootstrap_c_index(model, X_train, y_train)
    c_index_test, ci_lower_test, ci_upper_test = bootstrap_c_index(model, X_test, y_test)

    with open(os.path.join(output_dir, "concordance_index.txt"), "w") as f:
        f.write(f"Train Concordance Index: {c_index_train:.3f} ({ci_lower_train:.3f}-{ci_upper_train:.3f})\n")
        f.write(f"Test Concordance Index: {c_index_test:.3f} ({ci_lower_test:.3f}-{ci_upper_test:.3f})\n")
        f.write(f"Best Hyperparameters: {best_params}\n")
    
    experiment_result = {
        'experiment_name': folder_name,
        'train_c_index': c_index_train,
        'test_c_index': c_index_test,
        'train_c_index_ci': (ci_lower_train, ci_upper_train),
        'test_c_index_ci': (ci_lower_test, ci_upper_test),
        'km_plot': km_plot_path,
        'logrank_p_value': logrank_p,
        'biospecimen_embeddings_used': '_'.join(embedding_modalities),
        'methylation_embeddings_used': '_'.join(methylation_modalities),
        'clinical_used': '_'.join(other_data_modalities),
        'best_hyperparameters': best_params
    }

    clean_csv(folder_name)
    print("EXPERIMENT RESULT:", experiment_result)
    return experiment_result

def generate_experiments(results_dir, test_split, one_to_one, include_clinical_staging):
    os.makedirs(results_dir, exist_ok=True)
    
    # embedding_modalities = ["vit_DINO", "resnet50", "UNI_v1"]
    # methylation_modalities = ["ae_normalAE", "VAE_100k"]
    # other_data_modalities = ["clinical"]

    embedding_modalities = [[], ["vit_DINO"], ["resnet50"], ["UNI_v1"]]
    methylation_modalities = [[], ["ae_normalAE"], ["VAE_100k"]]
    other_data_modalities = [[], ["clinical"]]

    # embedding_modalities = [[], ["vit_DINO"], ["resnet50"], ["UNI_v1"], ["vit_DINO", "resnet50"], ["vit_DINO", "UNI_v1"], ["resnet50", "UNI_v1"], ["vit_DINO", "resnet50", "UNI_v1"]]
    # methylation_modalities = [[], ["ae_normalAE"], ["VAE_100k"], ["ae_normalAE", "VAE_100k"]]
    # other_data_modalities = [[], ["clinical"]]

    total_combinations = sum(1 for e in embedding_modalities for m in methylation_modalities for o in other_data_modalities if e or m or o)
    all_results = []

    with tqdm(total=total_combinations, desc="Running Experiments") as pbar:
        for embedding in embedding_modalities:
            for methylation in methylation_modalities:
                for other_data in other_data_modalities:
                    if not (embedding or methylation or other_data):  # Skip empty combinations
                        continue
                    if (not embedding and not methylation) and other_data:
                        pbar.update(1)
                        continue
                    experiment_result = run_experiment(embedding, methylation, other_data, results_dir, test_split, one_to_one, include_clinical_staging)
                    if(experiment_result is None):
                        print("Skipping")
                        pbar.update(1)
                        continue
                    all_results.append(experiment_result)
                    pbar.update(1)

    results_df = pd.DataFrame(all_results)
    results_df = results_df.sort_values(by="test_c_index", ascending=False)
    results_df.to_csv(os.path.join(results_dir, "results.csv"), index=False)

    print(f"All experiments completed. Results saved to {os.path.join(results_dir, 'results.csv')}")

# Small Dataset:
# generate_experiments("results_optuna_OTO_0.15_SPLIT", 0.15, True, False)
# generate_experiments("results_optuna_OTO_0.15_SPLIT_CLINICAL_STAGING", 0.15, True, True)
generate_experiments("results_optuna_OTO_0.30_SPLIT", 0.30, True, False)
generate_experiments("results_optuna_OTO_0.30_SPLIT_CLINICAL_STAGING", 0.30, True, True)

# Larger Dataset:
# generate_experiments("results_optuna_MTO_0.15_SPLIT", 0.15, True, False)
# generate_experiments("results_optuna_MTO_0.15_SPLIT_CLINICAL_STAGING", 0.15, True, True)
generate_experiments("results_optuna_MTO_0.30_SPLIT", 0.30, True, False)
generate_experiments("results_optuna_MTO_0.30_SPLIT_CLINICAL_STAGING", 0.30, True, True)

Running Experiments:   0%|          | 0/23 [00:00<?, ?it/s]

Using previous best hyperparams...
EXPERIMENT RESULT: {'experiment_name': 'embeddings__methylation_ae_normalAE_clinical_', 'train_c_index': 0.923057637596486, 'test_c_index': 0.6600665889477136, 'train_c_index_ci': (0.8114334865198584, 0.9875342245888329), 'test_c_index_ci': (0.4363303554982501, 0.8935937499999999), 'km_plot': 'results_optuna_OTO_0.30_SPLIT/embeddings__methylation_ae_normalAE_clinical_/km_curve.png', 'logrank_p_value': 0.4174933621160307, 'biospecimen_embeddings_used': '', 'methylation_embeddings_used': 'ae_normalAE', 'clinical_used': '', 'best_hyperparameters': {'n_estimators': 76, 'learning_rate': 0.041540343664469924, 'max_depth': 5, 'min_samples_split': 6}}
Using previous best hyperparams...
EXPERIMENT RESULT: {'experiment_name': 'embeddings__methylation_ae_normalAE_clinical_clinical', 'train_c_index': 0.9561972060195717, 'test_c_index': 0.647082432236093, 'train_c_index_ci': (0.8672393451822942, 0.9932297486000148), 'test_c_index_ci': (0.30941358024691357, 0.91937

[I 2025-04-03 05:05:18,525] A new study created in memory with name: no-name-c356b885-53e8-4440-9f35-8f145f4d3ae5
[I 2025-04-03 05:05:20,440] Trial 1 finished with value: 0.5889677628408603 and parameters: {'n_estimators': 28, 'learning_rate': 0.07243420468746845, 'max_depth': 4, 'min_samples_split': 2}. Best is trial 1 with value: 0.5889677628408603.
[I 2025-04-03 05:05:21,448] Trial 2 finished with value: 0.6364454768951113 and parameters: {'n_estimators': 56, 'learning_rate': 0.01744615099127261, 'max_depth': 3, 'min_samples_split': 7}. Best is trial 2 with value: 0.6364454768951113.
[I 2025-04-03 05:05:21,569] Trial 3 finished with value: 0.5832114635579408 and parameters: {'n_estimators': 74, 'learning_rate': 0.1451117075766948, 'max_depth': 2, 'min_samples_split': 9}. Best is trial 2 with value: 0.6364454768951113.
[I 2025-04-03 05:05:21,946] Trial 0 finished with value: 0.6333590553999955 and parameters: {'n_estimators': 91, 'learning_rate': 0.01574877191372956, 'max_depth': 2, 

EXPERIMENT RESULT: {'experiment_name': 'embeddings_vit_DINO_methylation_ae_normalAE_clinical_', 'train_c_index': 0.976225348622538, 'test_c_index': 0.7042454846193057, 'train_c_index_ci': (0.936417268440056, 0.9955599615477049), 'test_c_index_ci': (0.4689176414982867, 0.8675476937937695), 'km_plot': 'results_optuna_OTO_0.30_SPLIT/embeddings_vit_DINO_methylation_ae_normalAE_clinical_/km_curve.png', 'logrank_p_value': 0.0787014666178646, 'biospecimen_embeddings_used': 'vit_DINO', 'methylation_embeddings_used': 'ae_normalAE', 'clinical_used': '', 'best_hyperparameters': {'n_estimators': 53, 'learning_rate': 0.034830930443750456, 'max_depth': 4, 'min_samples_split': 3}}


[I 2025-04-03 05:05:39,005] A new study created in memory with name: no-name-19818b2d-f3d0-488f-875e-8ad3d006620b
[I 2025-04-03 05:05:40,180] Trial 0 finished with value: 0.6127005546717449 and parameters: {'n_estimators': 30, 'learning_rate': 0.10094111991178847, 'max_depth': 2, 'min_samples_split': 2}. Best is trial 0 with value: 0.6127005546717449.
[I 2025-04-03 05:05:40,527] Trial 2 finished with value: 0.6749951321532203 and parameters: {'n_estimators': 26, 'learning_rate': 0.07376021868053753, 'max_depth': 3, 'min_samples_split': 3}. Best is trial 2 with value: 0.6749951321532203.
[I 2025-04-03 05:05:40,914] Trial 4 finished with value: 0.6829964805115523 and parameters: {'n_estimators': 29, 'learning_rate': 0.024450322248503926, 'max_depth': 4, 'min_samples_split': 3}. Best is trial 4 with value: 0.6829964805115523.
[I 2025-04-03 05:05:41,757] Trial 5 finished with value: 0.6336764783032982 and parameters: {'n_estimators': 33, 'learning_rate': 0.021782408376481675, 'max_depth': 

...

[I 2025-04-03 05:37:18,230] Trial 3 finished with value: 0.45690147034935036 and parameters: {'n_estimators': 28, 'learning_rate': 0.024625447607992852, 'max_depth': 3, 'min_samples_split': 10}. Best is trial 3 with value: 0.45690147034935036.
[I 2025-04-03 05:37:19,677] Trial 1 finished with value: 0.46307492464194966 and parameters: {'n_estimators': 47, 'learning_rate': 0.12906861490974103, 'max_depth': 2, 'min_samples_split': 6}. Best is trial 1 with value: 0.46307492464194966.
[I 2025-04-03 05:37:21,141] Trial 6 finished with value: 0.44456758567149085 and parameters: {'n_estimators': 32, 'learning_rate': 0.03404790314842952, 'max_depth': 4, 'min_samples_split': 4}. Best is trial 1 with value: 0.46307492464194966.
[I 2025-04-03 05:37:21,812] Trial 5 finished with value: 0.40618208386837806 and parameters: {'n_estimators': 44, 'learning_rate': 0.04852035235613868, 'max_depth': 3, 'min_samples_split': 2}. Best is trial 1 with value: 0.46307492464194966.
[I 2025-04-03 05:37:24,196] Tr

Maximum retries reached. Experiment failed.
Skipping
Using previous best hyperparams...
EXPERIMENT RESULT: {'experiment_name': 'embeddings_UNI_v1_methylation__clinical_', 'train_c_index': 0.9849462716387636, 'test_c_index': 0.7519118841535618, 'train_c_index_ci': (0.97404414875064, 0.9934820285457638), 'test_c_index_ci': (0.49174730236922354, 0.9057971014492754), 'km_plot': 'results_optuna_OTO_0.30_SPLIT/embeddings_UNI_v1_methylation__clinical_/km_curve.png', 'logrank_p_value': 0.053863471895339646, 'biospecimen_embeddings_used': 'UNI_v1', 'methylation_embeddings_used': '', 'clinical_used': '', 'best_hyperparameters': {'n_estimators': 100, 'learning_rate': 0.13539127556330524, 'max_depth': 2, 'min_samples_split': 8}}
Using previous best hyperparams...
EXPERIMENT RESULT: {'experiment_name': 'embeddings_UNI_v1_methylation__clinical_clinical', 'train_c_index': 0.9834461037368524, 'test_c_index': 0.7071797413629675, 'train_c_index_ci': (0.969668213038768, 0.9927764175437256), 'test_c_index

Running Experiments:   0%|          | 0/23 [00:00<?, ?it/s]

Using previous best hyperparams...
EXPERIMENT RESULT: {'experiment_name': 'embeddings__methylation_ae_normalAE_clinical_', 'train_c_index': 0.9119148720323276, 'test_c_index': 0.6965109355798731, 'train_c_index_ci': (0.7653379753152773, 0.9852329586304723), 'test_c_index_ci': (0.5155402260638298, 0.8576241346383385), 'km_plot': 'results_optuna_OTO_0.30_SPLIT_CLINICAL_STAGING/embeddings__methylation_ae_normalAE_clinical_/km_curve.png', 'logrank_p_value': 0.06381151280357822, 'biospecimen_embeddings_used': '', 'methylation_embeddings_used': 'ae_normalAE', 'clinical_used': '', 'best_hyperparameters': {'n_estimators': 94, 'learning_rate': 0.04670982543523739, 'max_depth': 4, 'min_samples_split': 4}}
Using previous best hyperparams...
EXPERIMENT RESULT: {'experiment_name': 'embeddings__methylation_ae_normalAE_clinical_clinical', 'train_c_index': 0.9775584776640546, 'test_c_index': 0.7272846407144097, 'train_c_index_ci': (0.9542084705457995, 0.9940659757330637), 'test_c_index_ci': (0.5375130

KeyboardInterrupt: 